In [ ]:
# Python data
 
import json
import presentation_functions as functions

building = 'ac_istzustand'
size = 'dez'
available_devices = ["HPC","HP", "BOI", "EB", "CHP", "BCHP", "PV", "STC", "TES"]

# Load the data
with open(f'../results/{building}.json') as f:
    results = json.load(f)





### Verfügbare Anlagen & Technologien

Für das Optimierungsproblem standen die folgenden Systeme zur Verfügung:

|Abkürzung|Näme (EN)|Name (DE)|Kosten (€/kW)|
|---|---|---|:---|
|HP|Heat Pump|Wärmepumpe|$1358$|
|BOI|Boiler|Boiler|$290$|
|EB|Electric Boiler|Elektroboiler|$188$|
|CHP|Combined Heat and Power|Heizkräftwerk|$3744$|
|BCHP|Bio Combined Heat and Power|Bio-Heizkräftwerk|$5000$|
|PV|Photovoltaic|Photovoltaik|$1314$|
|STC|Solar Thermal Collector|Solarthermie|$696$|
|TES|Thermal Energy Storage|Wärmespeicher|$9200 \ \text{m}^3$/€ |



### Technische Beschränkungen

Die folgenden technischen Beschränkungen sind zu beachten:
1. Jedes Gebäude hat eine andere Dachfläche (und daher muss die kombinierte PV- und STC-Größe kleiner sein als diese)
   1. Aachener Grundvermögen Gebäude: $152  \ \text{m}^2$
   2. Privates Mehrfamilienhaus: $123 \ \text{m}^2$
   3. Hein-Nixdorf-Berufskolleg: $1600 \ \text{m}^2$
   4. Sparkasse Essen: $0 \ \text{m}^2$


### Jährliche Bedarfe

In [ ]:
# 📊 Führen die Python-Zelle aus, um die Diagramme zu sehen

functions.jährliche_bedarfe(results)


## Anlagen

### Kapazität

Alle Anlagen haben eine bestimmte **Kapazität**, die im Prinzip die Größe des Anlages angibt. Das Gerät kann pro Stunde nicht mehr Energie erzeugen als seine Kapazität beträgt. Für das **Netz** ist die Kapazität die maximale Energie, die in einer Stunde übertragen wird (import oder export).

In [ ]:
# 📊 (Kapazität) Führen die Python-Zelle aus, um die Diagramme zu sehen

functions.plot_kapazitaten(results)

In [ ]:
# (Andere daten, Volum, Fläche, Curtailed & Mean COP) Führen die Python-Zelle aus, um die Diagramme zu sehen

for device in results["Devices"].keys():
    if device in ["TES", "BAT", "CTES"]:
        print(f'{device} Volum: {results["Devices"][device]["Volume"]} m3')
    elif device in ["STC", "PV"]:
        print(f'{device} Fläche: {results["Devices"][device]["Area"]} m2')
    if device in ["STC", "WAT", "PV", "WT"]:
        print(f'{device} curtailed: {results["Devices"][device]["Curtailed"]} kWh')
    if device in ["HP", "GHP"]:
        print(f'{device} mean COP: {results["Devices"][device]["Mean COP"]}')

### Stündliche Ergebnisse

Im folgenden Diagramm sind die stündlichen Daten für jedes Gerät zu sehen (inkl. Netz import und export).

Die folgenden Daten sind zu sehen:
- Empfangene/erzeugte Energie: z.B., wie viel Wärme ein HP in einer Stunde erzeugt hat und wie viel Strom es empfangen hat. Die empfangende Energie stammt aus dem gemeinsamen Strompool (also dem Netz und den Stromerzeugerengeräte).
- Ladezustand (für BAT oder TES)
- Ladung/Entladung (für BAT oder TES)

Wählen Sie die Parameter und dann beide Zellen (zuerst die Parameterzelle und dann die Diagrammzelle)
- `devices`: Ein oder mehrere Anlagen (um Netzdaten zu sehen: "import" oder "export" als Anlagename)
- `types`: Ein oder mehrere von "Generated", "Received", "State of Charge" oder "Charged/Discharged"
- `sub_types`: Ein oder mehrere von "Heat", "Electricity" oder "Cooling"
- `hour_range`: Für welchen Stundenbereich der Medianwert zu nehmen ist. 
  - z.B nimmt [4,10] den Medianwert der Daten von 04:00 bis 10:00
- `day_rage`: Für welchen Tagesbereich das Diagramm erstellt werden soll.
  - z.B. zeigt [0,10] die Mittelwerte vom 1. Januar bis zum 11. Januar

In [ ]:
# Parametern

devices = ["PV", "import"]
types = ["Generated", "Received"]
sub_types = ["Heat", "Electricity"]
hour_range = [8, 16]
day_range = [120, 150]

In [ ]:
# 📊 (Mittelwert) Führen die Python-Zelle aus, um die Diagramme zu sehen

functions.plot_mittelwert(available_devices,building, devices, types, sub_types, hour_range, day_range)


### Erzeugte Energie

Die erzeugte Energie (Strom/Wärme oder Kühlung) ist die gesamte vom Gerät erzeugte Energie, einschließlich der Energie, die in die Versorgung anderer Geräte fließt.

Die Einfuhr aus dem Netz bedeutet also nicht, dass sie direkt vom Endverbraucher (d. h. den dort lebenden Personen) genutzt wird, sondern sie kann auch von anderen Anlagen verwendet werden (z. B. von HP zur Wärmeerzeugung).

Alle Energiearten (Wärme, Kälte oder Strom) kommen zunächst in einen gemeinsamen Pool, und von dort wird berechnet, wie viel davon in die Analgen fließt, wobei immer genug übrig bleibt, um den Bedarf zu decken.

In [ ]:
# 📊 Führen Sie die Python-Zelle aus, um das Ergebnis zu sehen

functions.plot_erzeugte_energie(results)

## Kosten

In [ ]:
# (Gesamtkosten) Führen Sie die Python-Zelle aus, um das Ergebnis zu sehen

total_payed = + results["Total Costs"]["Total device costs"]["Total investment costs"] + results["Total Costs"]["Total device costs"]["Total O&M costs"] + results["Total Costs"]["Total device costs"]["Total demand related costs"] + results["Total Costs"]["CO2 Tax"] + results["Total Costs"]["Supply costs"]["Electricity"]["Supply costs"] + results["Total Costs"]["Supply costs"]["Electricity"]["Cap costs"] + results["Total Costs"]["Supply costs"]["Gas"]["Supply costs"] + results["Total Costs"]["Supply costs"]["Gas"]["Cap costs"] + results["Total Costs"]["Supply costs"]["Heat"]["Supply costs"] + results["Total Costs"]["Supply costs"]["Heat"]["Cap costs"] + results["Total Costs"]["Supply costs"]["Biomass"] + results["Total Costs"]["Supply costs"]["Waste"] + results["Total Costs"]["Supply costs"]["Hydrogen"]


total_revenue = results["Total Costs"]["Supply costs"]["Electricity"]["Feed-in revenues"] + results["Total Costs"]["Supply costs"]["Gas"]["Feed-in revenues"] + results["Total Costs"]["Supply costs"]["Heat"]["Feed-in revenues"]

total_annualized_cost = total_payed - total_revenue

print(f"(=) Gesamtkosten (annualized) : {total_annualized_cost} €/a")   
print(f"(+) Insgesamt gezahlt         : {total_payed} €/a")
print(f"(-) Einahmen (feed-in)        : {total_revenue} €/a")


### Kosten nach Gerät

In [ ]:
# 📊 (Anlagekosten) Führen Sie die Python-Zelle aus, um das Ergebnis zu sehen

functions.kosten_nach_gerät(results)

### Kosten nach Art

In [ ]:
# 📊 (Kosten nach Art) Führen Sie die Python-Zelle aus, um das Ergebnis zu sehen

functions.kosten_art(results)